In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import numpy as np
from glob import glob as gl
import config as cf
import imutils


W = 224
H = 224

In [ ]:
def make_model():
    inputs = keras.Input(shape=(H, W, 3))

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    # x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        # x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        # x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(512, 3, padding="same")(x)
    # x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(1,  activation='linear')(x)
    return keras.Model(inputs, x)



In [ ]:
def padding_to_square(img, W, H):
    h, w = img.shape[:2]
    out = np.zeros((H, W, 3), np.uint8)
    r = min(W/w, H/h)
    img = cv2.resize(img, (int(r*w), int(r*h)))
    h, w = img.shape[:2]
    out[(H-h)//2:(H-h)//2+h, (W-w)//2:(W-w)//2+w] = img
    return out

def make_data(path):
    images = []
    labels = []
    files = gl(path+"/*.jpg")
    for file in files:
        try:
            label_file = file[:-3]+"txt"
            f = open(label_file, 'r')
            a = f.readline()
            angle = float(a)
            img = cv2.imread(file, 1)
            img = padding_to_square(img, W, H)
            # img = np.expand_dims(img, axis=2)
            images.append(img)
            labels.append(angle)
        except Exception as e:
            pass
    return np.array(images), np.array(labels)


In [ ]:
images, lables = make_data("aug")
print(images.shape, lables.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, lables, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
k = 0.8

In [ ]:
epochs = 20
batch_size = 16

model = make_model()
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="mse"
)
# model.summary()
model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size = batch_size,
    validation_data = (X_test, y_test)
)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
predict = model.predict(X_test)
for i in range(X_test.shape[0]):
    print(predict[i], y_test[i])

In [ ]:
model.save("model.h5")